# 📚 Kapitola 26: Performance optimalizace

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 3 | ADVANCED</h2>
    <p style="color: white; margin: 10px 0;">📖 Výuková kapitola</p>
</div>

## 🎯 Co se naučíte

V této kapitole se zaměříme na následující témata:

- **Python profiling tools**
- **Memory profiling**
- **Caching strategies**
- **Redis implementation**
- **Database query optimization**
- **Async vs. sync performance**
- **Load testing with locust**

## ⚠️ Předpoklady
Tato kapitola navazuje na kapitoly: 17

---


In [ ]:
# ⚙️ Inicializace prostředí
# Tento kód nastavuje prostředí pro kapitolu 26

import sys
import os
from datetime import datetime

# Informace o prostředí
print("=" * 60)
print(f"📚 KAPITOLA 26: Performance optimalizace")
print("=" * 60)
print(f"🐍 Python verze: {sys.version}")
print(f"📅 Datum spuštění: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💻 OS: {os.name}")
print(f"📁 Pracovní adresář: {os.getcwd()}")
print("=" * 60)

# Instalace potřebných knihoven (odkomentujte podle potřeby)
# !pip install requests pandas numpy matplotlib
# !pip install beautifulsoup4 sqlalchemy fastapi

# Import základních knihoven
import json
import csv
import time
import random
from pathlib import Path
from typing import List, Dict, Optional, Any

print("✅ Prostředí připraveno!")

## 📖 Teoretická část

<div style="background: #f0f4ff; padding: 20px; border-left: 5px solid #4a69bd; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🎓 Základní teorie a koncepty</h3>
</div>

# Performance optimalizace

## 1. Úvod a motivace

Performance optimalizace je klíčová oblast, kterou každý vývojář snaží dosáhnout v praxi – zvlášť při tvorbě aplikací, které se musí vyrovnat s nárůstem počtu uživatelů a zvýšenými nároky na rychlost. V dnešním prostředí, kdy uživatelé očekávají okamžité odezvy, je optimální výkon aplikace mnohdy rozhodujícím faktorem pro její úspěch. Tato kapitola se zaměřuje na pokročilé techniky a nástroje, které umožní zvýšit rychlost, efektivitu a stabilitu aplikací – od jednoduchých profilovacích nástrojů až po pokročilé strategie ukládání do mezipaměti nebo optimalizaci databázových dotazů.

Tento obsah je přímo aplikovatelný v praxi – včetně implementace Redis, nástrojů pro profilování a optimalizací dotazů. Student se naučí analyzovat výkonnost aplikací, identifikovat zásadní příčiny zpomalení a implementovat efektivní řešení.

---

## 2. Hlavní koncepty

### Python profiling tools

Profiling je proces, který pomáhá zjistit, kde se v programu nejvíce času potrápí – tedy kde jsou "bottlenecky". V Pythonu existuje několik nástrojů pro profiling:

- **cProfile** – zabudovaný modul, který generuje detailní profilovací data ve formátu tabulky.
```bash
python -m cProfile -o profile_output.prof my_script.py
```
Použití `pstats` pro analýzu výsledků:
```python
import pstats
stats = pstats.Stats('profile_output.prof')
stats.sort_stats('cumulative').print_stats(10)
```

- **line_profiler** – nástroj, který ukazuje rychlost provádění jednotlivých řádků kódu:
```bash
pip install line_profiler
kernprof -l -v my_script.py
```

- **py-spy** – externí nástroj pro profilování běžících Python procesů, ideální pro produkční prostředí.

Profilování je zásadní pro identifikaci kritických částí aplikace, kde se výkon nezvyšuje, ale naopak zhoršuje.

---

### Memory profiling

Využití paměti (memory usage) je stejně důležité jako rychlost provádění – ztráta paměti může vést k výpadkům. V Pythonu se používají nástroje:

- **memory_profiler** – umožňuje sledovat spotřebu paměti na úrovni řádků:
```bash
pip install memory_profiler
python -m memory_profiler my_script.py
```

- **tracemalloc** – zabudovaný modul pro sledování alokace paměti v reálném čase.

Příklad:
```python
import tracemalloc

tracemalloc.start()
# nějaký kód
current, peak = tracemalloc.get_traced_memory()
print(f"Aktuální: {current / 1024 / 1024} MB")
print(f"Nejvyšší: {peak / 1024 / 1024} MB")
tracemalloc.stop()
```

Znížení memory leaků a efektivní správa paměti je klíčové pro dlouhodobou stabilitu systému.

---

### Caching strategies

Caching je způsob, jak ušetřit čas tím, že se výpočty neopakují. V Pythonu se používají různé strategie:

- **LRU Cache (Least Recently Used)** – např. dekorátor `@lru_cache(maxsize=128)` z modulu `functools`.
```python
from functools import lru_cache

@lru_cache(maxsize=128)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)
```

- **Custom caching** – např. pomocí `dict` nebo Redis, pokud je potřeba cache na více serverech.

Strategie cache se volí podle typu dat a způsobu jejich využití – např. data, která se často opakují, jsou vhodná pro caching.

---

### Redis implementation

Redis je velmi rychlý in-memory databázový systém, který se používá jako cache, fronta nebo databáze. V Pythonu se připojuje pomocí `redis-py`:

```bash
pip install redis
```

Příklad:
```python
import redis

r = redis.Redis(host='localhost', port=6379, db=0)
r.set('key', 'value')
print(r.get('key'))
```

Redis je ideální pro ukládání nečastých dat (např. session), rychlé vyhledávání, nebo jako mezipaměť pro databáze.

---

### Database query optimization

Databázové dotazy jsou často největším bottleneckem v aplikacích. Optimalizace zahrnuje:

- **Indexování** – přidání indexů na často dotahované sloupce.
```sql
CREATE INDEX idx_user_email ON users(email);
```

- **EXPLAIN** – použití `EXPLAIN` pro analýzu dotazu:
```sql
EXPLAIN SELECT * FROM users WHERE email = 'user@example.com';
```

- **Lazy loading** a **batching** – např. při práci s ORM (např. Django, SQLAlchemy) se vyhýbeš náročným JOINům.

- **Query caching** – cacheování výsledků častých dotazů.

---

### Async vs. sync performance

Synchronní programy blokují hlavní vlákno, zatímco asynchronní (async) kód umožňuje provádění více operací současně – ideální pro I/O operace:

```python
import asyncio

async def fetch_data(url):
    # Např. HTTP request
    pass

async def main():
    tasks = [fetch_data(url) for url in urls]
    await asyncio.gather(*tasks)

asyncio.run(main())
```

Async kód je efektivní pro zpracování mnoha I/O omezených operací (např. API volání, databázové dotazy). Neefektivní je pro CPU náročné úkoly.

---

### Load testing with Locust

Locust je open-source nástroj pro load testing. Umožňuje simulovat mnoho uživatelů a testovat, jak se systém chová pod zátěží:

```python
from locust import HttpUser, task, between

class WebsiteUser(HttpUser):
    wait_time = between(1, 3)

    @task
    def index(self):
        self.client.get("/")
```

Spuštění:
```bash
locust -f locustfile.py
```

Pomocí Locustu můžeš určit, kolik uživatelů systém zvládne zvládnout, a najít body selhání.

---

## 3. Důležité detaily

### Časté chyby a jak se jim vyhnout:

- **Nepoužívání cache** – při opakovaném výpočtu stejných dat.
- **Zbytečné JOINy** – zpomalují dotazy.
- **Použití sync funkcí pro I/O operace** – zpomaluje aplikaci.
- **Nelokální cache** – např. výpočty ve více procesech bez sdílení dat.

### Best practices:

- Vždy testuj výkon na reálných datech.
- Profiluj před a po optimalizacích.
- Používej cache pro častěji využívaná data.
- Optimalizuj dotazy, nejdříve indexuj, poté se podívej na strukturu.

### Performance tipy:

- Zvaž použití `__slots__` pro třídy s mnoha instancemi.
- Využívej generátory místo seznamů, pokud není nutné uchovávat všechna data.
- Používej `multiprocessing` pro CPU náročné úkoly, `asyncio` pro I/O.

---

## 4. Propojení s předchozími kapitolami

Tato kapitola navazuje na předchozí znalosti o Pythonu, databázích a architektuře aplikací. V rámci těchto poznatků jsme se naučili vytvářet funkcionalitu, ale nyní se zaměřujeme na to, jak zvýšit jejich efektivitu. Např. pokud již víš, jak pracovat s ORM (např. SQLAlchemy), tato kapitola ti ukáže, jak optimalizovat dotazy a využít cache. Důležité je také porozumět principům asynchronního programování, které jsou klíčem k efektivnímu zpracování paralelních operací.

---

Závěrem: Performance optimalizace je nezbytná doba pro každého profesionálního vývojáře. Tato kapitola tě naučí mít přehled o nástrojích, které ti pomohou identifikovat a odstranit zpomalení – a to jak na úrovni kódu, tak i celkového systému.


## 💻 Praktické příklady

<div style="background: #e8f5e9; padding: 20px; border-left: 5px solid #4caf50; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">👨‍💻 Hands-on příklady ke spuštění</h3>
    <p>Následující příklady si můžete hned vyzkoušet. Každý příklad je samostatně spustitelný.</p>
</div>


### Příklad 1

### Co příklad demonstruje:
Tento příklad ukazuje základní použití modulu `cProfile` k profilování času běhu Python programu a identifikaci nejpomalejších funkcí.

In [ ]:
import cProfile
import math

def slow_function():
    """Pomalá funkce pro profilování"""
    total = 0
    for i in range(1000000):
        total += math.sqrt(i)
    return total

def fast_function():
    """Rychlá funkce"""
    return sum(range(1000))

# Profilování spuštění funkcí
if __name__ == "__main__":
    print("Spouštím pomalou funkci...")
    cProfile.run('slow_function()')
    
    print("\nSpouštím rychlou funkci...")
    cProfile.run('fast_function()')

### Příklad 2

### Očekávaný výstup:
Výstup bude obsahovat časové statistiky pro jednotlivé funkce, například:

```
         1000003 function calls in 1.254 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.254    1.254 profiling_example.py:5(slow_function)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.sum}
        1    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
```

### Příklad 3

### Vysvětlení:
Modul `cProfile` profiluje, kolik času trvá každá funkce a jak často se volá. Výstup ukazuje počet volání (`ncalls`), celkový čas (`tottime`) a kumulativní čas (`cumtime`). Pomocí toho lze identifikovat, které části programu jsou nejvíce zatěžující.

---

# 2. **Memory Profiling - sledování spotřeby paměti**

### Příklad 4

### Co příklad demonstruje:
Tento příklad používá `memory_profiler` k monitorování využití paměti během běhu programu.

In [ ]:
from memory_profiler import profile

@profile
def memory_intensive_function():
    """Funkce, která spotřebovává hodně paměti"""
    # Vytvoření velkého seznamu
    large_list = [i for i in range(1000000)]
    
    # Přidání dalších dat do seznamu
    large_list.extend([i**2 for i in range(500000)])
    
    return sum(large_list)

if __name__ == "__main__":
    result = memory_intensive_function()
    print(f"Výsledek: {result}")

### Příklad 5

### Očekávaný výstup:
Výstup bude obsahovat informace o paměti spotřebované jednotlivými řádky kódu:

```
Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
     3     47.5 MiB     47.5 MiB           1   @profile
     4                                         def memory_intensive_function():
     5     60.2 MiB     12.7 MiB           1       large_list = [i for i in range(1000000)]
     6     92.3 MiB     32.1 MiB           1       large_list.extend([i**2 for i in range(500000)])
     7     92.3 MiB      0.0 MiB           1       return sum(large_list)
```

### Příklad 6

### Vysvětlení:
`memory_profiler` ukazuje, jak se mění využití paměti při provádění jednotlivých řádků kódu. Pomocí `@profile` dekorátoru lze snadno zjistit, které části programu spotřebovávají nejvíce paměti.

---

# 3. **Caching Strategies - využití functools.lru_cache**

### Příklad 7

### Co příklad demonstruje:
Tento příklad ukazuje jednoduchou implementaci mezipaměti pomocí `functools.lru_cache` pro zvýšení rychlosti opakovaných výpočtů.

In [ ]:
import time
from functools import lru_cache

@lru_cache(maxsize=128)
def fibonacci(n):
    """Funkce výpočtu Fibonacciho posloupnosti s mezipamětí"""
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

def benchmark():
    """Benchmark pro porovnání rychlosti bez a s cache"""
    start_time = time.time()
    result = fibonacci(35)
    end_time = time.time()
    
    print(f"Fibonacci(35) = {result}")
    print(f"Čas běhu: {end_time - start_time:.4f} sekund")

if __name__ == "__main__":
    benchmark()

### Příklad 8

### Očekávaný výstup:
```
Fibonacci(35) = 9227465
Čas běhu: 0.0001 sekund
```

### Příklad 9

### Vysvětlení:
`@lru_cache` ukládá výsledky posledních volání funkce do mezipaměti. Pokud se stejný vstup opakuje, funkce vrátí výsledek z paměti místo opětovného výpočtu. Tím se sníží čas běhu pro opakované volání.

---

# 4. **Redis Implementation - cache v Redisu**

### Příklad 10

### Co příklad demonstruje:
Tento příklad ukazuje, jak používat Redis jako mezipaměť pro rychlé načítání dat.

In [ ]:
import redis
import time

# Připojení k Redis serveru (předpokládáme, že běží lokálně)
r = redis.StrictRedis(host='localhost', port=6379, db=0)

def get_data_from_cache(key):
    """Získání dat z Redis cache"""
    cached_data = r.get(key)
    if cached_data:
        print(f"Načteno z cache: {cached_data.decode()}")
        return cached_data.decode()
    else:
        # Simulace pomalého výpočtu
        time.sleep(1)
        data = f"Data pro klíč {key}"
        r.setex(key, 60, data)  # Uložení do cache na 60 sekund
        print(f"Vypočteno a uloženo do cache: {data}")
        return data

if __name__ == "__main__":
    print("První požadavek (bude trvat 1 sekundu):")
    get_data_from_cache('user_123')
    
    print("\nDruhý požadavek (převezme z cache):")
    get_data_from_cache('user_123')

### Příklad 11

### Očekávaný výstup:
```
První požadavek (bude trvat 1 sekundu):
Vypočteno a uloženo do cache: Data pro klíč user_123

Druhý požadavek (převezme z cache):
Načteno z cache: Data pro klíč user_123
```

### Příklad 12

### Vysvětlení:
Redis slouží jako mezipaměť, kde se data ukládají na určitou dobu (`setex`). Při následném požadavku se data načtou z Redisu místo opětovného výpočtu. Zrychluje tím rychlost odpovědi.

---

# 5. **Database Query Optimization - použití EXPLAIN a indexů**

### Příklad 13

### Co příklad demonstruje:
Tento příklad ukazuje, jak optimalizovat databázové dotazy pomocí indexů a `EXPLAIN`.

In [ ]:
import sqlite3
import time

# Vytvoření spojení s databází SQLite
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# Vytvoření tabulky
cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT UNIQUE,
    age INTEGER
)""")

# Vložení testovacích dat
cursor.execute("INSERT OR IGNORE INTO users (id, name, email, age) VALUES (1, 'Petr', 'petr@example.com', 30)")
cursor.execute("INSERT OR IGNORE INTO users (id, name, email, age) VALUES (2, 'Jana', 'jana@example.com', 25)")

# Vytvoření indexu na sloupec `email`
cursor.execute("CREATE INDEX IF NOT EXISTS idx_email ON users(email)")

conn.commit()

def query_with_explain():
    """Dotaz s EXPLAIN pro analýzu"""
    print("Bez indexu:")
    cursor.execute("EXPLAIN QUERY PLAN SELECT * FROM users WHERE email = 'petr@example.com'")
    print(cursor.fetchall())
    
    # Výpočet času běhu dotazu
    start_time = time.time()
    cursor.execute("SELECT * FROM users WHERE email = 'petr@example.com'")
    result = cursor.fetchone()
    end_time = time.time()
    
    print(f"Výsledek: {result}")
    print(f"Čas běhu: {end_time - start_time:.6f} sekund")

if __name__ == "__main__":
    query_with_explain()

### Příklad 14

### Očekávaný výstup:
```
Bez indexu:
[(0, 0, 0, 'SCAN TABLE users', None, None, None, None, None, None)]

Výsledek: (1, 'Petr', 'petr@example.com', 30)
Čas běhu: 0.000

## 🎯 Cvičení a úkoly

<div style="background: #fff3e0; padding: 20px; border-left: 5px solid #ff9800; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">✍️ Praktická cvičení k procvičení</h3>
    <p>Vyřešte následující úkoly. Začněte od jednoduššších a postupujte k složitějším.</p>
</div>

# Cvičení pro kapitolu: Performance optimalizace

---

## 1. **Název úkolu**  
Optimalizace databázového dotazu s využitím indexů a EXPLAIN

### 2. **Detailní zadání**
Student dostane tabulku `orders` s více než 100 000 řádky, která obsahuje sloupce: `id`, `customer_id`, `order_date`, `amount`. Musí optimalizovat dotaz, který vybírá všechny objednávky z určitého roku a seřadí je podle částky. Student musí:
- Změnit strukturu dotazu (např. přidání indexu)
- Porovnat výkonnost s a bez indexu
- Využít `EXPLAIN` pro analýzu

### 3. **Vstupní data/požadavky**
- Tabulka `orders` s 100 000 řádky
- Sloupce: `id`, `customer_id`, `order_date`, `amount`
- Dotaz: `SELECT * FROM orders WHERE YEAR(order_date) = 2022 ORDER BY amount DESC;`

### 4. **Očekávaný výstup**
- Výsledný dotaz s indexem a jeho výkonová analýza (EXPLAIN)
- Změna času provádění
- Vysvětlení, proč byl dotaz zpomalena bez indexu

### 5. **Hints/Nápověda**
1. Nejprve spusťte `EXPLAIN` na původní dotaz a zjistěte, jaký typ vyhledávání používá.
2. Vytvořte index na sloupec `order_date` (nebo `YEAR(order_date)`, pokud není možné vytvořit index na funkci).
3. Přepište dotaz tak, aby využíval přímo datum místo funkce `YEAR`.
4. Znovu spusťte `EXPLAIN` a porovnejte výsledky.

### 6. **Kostra řešení**
```sql
-- Původní dotaz
SELECT * FROM orders WHERE YEAR(order_date) = 2022 ORDER BY amount DESC;

-- Optimalizovaný dotaz
SELECT * FROM orders WHERE order_date BETWEEN '2022-01-01' AND '2022-12-31' ORDER BY amount DESC;

-- Vytvoření indexu
CREATE INDEX idx_orders_date ON orders(order_date);
```

### 7. **Bonusové rozšíření**
Vytvořte složitější dotaz, který využívá více tabulek s JOINy a optimalizujte ho pomocí indexů a `EXPLAIN`.

---

## 2. **Název úkolu**  
Profiling aplikace s využitím nástrojů pro měření výkonu

### 2. **Detailní zadání**
Student má přístup ke zdrojovému kódu jednoduchého backendového API (např. Python + Flask). Musí:
- Spustit profiler a analyzovat čas provádění funkcí
- Identifikovat největší přírůstky výkonu
- Optimalizovat problematické funkce (např. cyklus, opakované SQL dotazy)
- Změřit zlepšení po optimalizaci

### 3. **Vstupní data/požadavky**
- Jednoduchá aplikace v Pythonu s Flaskem
- Endpoint, který volá několik funkcí a může provádět SQL dotazy
- Nástroje: `cProfile`, `line_profiler`, `py-spy`

### 4. **Očekávaný výstup**
- Profiling report (např. výstup z `cProfile`)
- Seznam nejpomalejších funkcí
- Optimalizovaný kód a měření výkonnosti po optimalizaci

### 5. **Hints/Nápověda**
1. Spusťte `cProfile.run('app.run()')` pro získání profilovacích dat.
2. Použijte `line_profiler` pro řádkový profiling funkcí.
3. Vyhledejte funkce s vysokým časem provádění, např. cykly nebo opakované SQL dotazy.
4. Optimalizujte tuto část kódu a znovu spusťte profiler.

### 6. **Kostra řešení**
```python
import cProfile
from flask import Flask

app = Flask(__name__)

@app.route('/slow')
def slow_function():
    # Nějaký pomalý kód
    return "Hello"

if __name__ == '__main__':
    cProfile.run('app.run()')
```

### 7. **Bonusové rozšíření**
Vytvořte mikroslužbu s více endpointy a optimalizujte výkonnost pomocí async/await nebo Redis cache.

---

## 3. **Název úkolu**  
Cache implementace s Redisem – optimalizace opakovaných volání

### 2. **Detailní zadání**
Student dostane aplikaci, která provádí několik SQL dotazů pro každý požadavek. Musí:
- Přidat Redis cache pro úložiště výsledků
- Vytvořit logiku pro kontrolu cache před SQL dotazem
- Změřit zlepšení času odpovědi a využití databáze

### 3. **Vstupní data/požadavky**
- Aplikace s několika SQL dotazy (např. počet uživatelů, celkový počet objednávek)
- Databáze (např. PostgreSQL)
- Redis server

### 4. **Očekávaný výstup**
- Kód s implementovaným Redis cache
- Měření času odpovědi před a po cache
- Změna zatížení databáze

### 5. **Hints/Nápověda**
1. Pomocí `redis-py` vytvořte spojení s Redisem.
2. Uložte výsledky SQL dotazů do Redisu s klíčem (např. `cache:users_count`).
3. Před každým dotazem zkontrolujte, jestli klíč existuje v Redisu.
4. Použijte `TTL` pro automatické vypršení cache.

### 6. **Kostra řešení**
```python
import redis
import psycopg2

r = redis.Redis(host='localhost', port=6379, db=0)

def get_users_count():
    key = 'users_count'
    cached = r.get(key)
    if cached:
        return int(cached)
    
    conn = psycopg2.connect(...)
    cur = conn.cursor()
    cur.execute("SELECT COUNT(*) FROM users")
    count = cur.fetchone()[0]
    r.setex(key, 3600, count)  # 1 hodina
    return count
```

### 7. **Bonusové rozšíření**
Implementujte distribuovanou cache s Redis Cluster nebo využijte LRU cache pro řízení paměti.

---

## 4. **Název úkolu**  
Asynchronní volání API s optimalizací paralelismu

### 2. **Detailní zadání**
Student dostane seznam URL, které musí přistoupit a stáhnout data (např. JSON). Musí:
- Implementovat asynchronní volání pomocí `asyncio` nebo `aiohttp`
- Optimalizovat počet paralelních požadavků (např. pomocí semaforu)
- Změřit zlepšení času odpovědi

### 3. **Vstupní data/požadavky**
- Seznam URL (např. více API endpointů)
- Ka


## 📚 Další zdroje a materiály

<div style="background: #e3f2fd; padding: 20px; border-left: 5px solid #2196f3; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🔗 Doporučené materiály k dalšímu studiu</h3>
</div>

# Zdroje pro kapitolu: Performance optimalizace

## Články a tutoriály

1. **[Optimizing Web Performance](https://web.dev/learn/performance/)** – *Google Developers*  
   Komplexní výukový kurz od Googlu zaměřený na optimalizaci webových stránek, včetně základních i pokročilých technik jako lazy loading, kachlování a optimalizace zdrojů.

2. **[JavaScript Performance Best Practices](https://developer.mozilla.org/en-US/docs/Web/Performance)** – *Mozilla Developer Network (MDN)*  
   Oficiální dokumentace od MDN obsahující tipy a techniky pro vylepšení výkonu JavaScriptu v prohlížečích, např. vyhýbání se zbytečným reflowům.

3. **[Web Vitals: The New Google Core Web Vitals](https://web.dev/vitals/)** – *Google Developers*  
   Přehled nových metrik výkonu od Googlu, které mají vliv na SEO a uživatelský zážitek. Tento článek je ideální pro pochopení aktuálních indikátorů výkonu.

4. **[How to Optimize Images for the Web](https://css-tricks.com/optimizing-images-for-the-web/)** – *CSS-Tricks*  
   Detailní průvodce optimalizací obrázků pomocí různých formátů (WebP, AVIF), komprese a lazy loadingu.

5. **[Understanding React Performance Optimization](https://reactjs.org/docs/react-performance.html)** – *React Documentation*  
   Oficiální dokumentace o optimalizaci výkonu v React aplikacích, například pomocí `React.memo`, `useMemo`, `useCallback` a `lazy loading` komponent.

## YouTube videa

1. **[Web Performance Optimization (WPO) - Crash Course](https://www.youtube.com/watch?v=3F3592gH07U)** – *19 min*  
   Základní přehled optimalizace webového výkonu od vývojářské komunity, který zahrnuje rychlost načítání stránek, lazy loading a zpracování obrázků.

2. **[JavaScript Performance Tips & Tricks](https://www.youtube.com/watch?v=VQ341s3W5ZI)** – *25 min*  
   Video se zaměřuje na konkrétní tipy pro vylepšení rychlosti JavaScriptového kódu, včetně optimalizací cyklů a práce s DOM.

3. **[Performance Testing in React Apps](https://www.youtube.com/watch?v=7J9F5wW0dZg)** – *22 min*  
   Toto video ukazuje praktické přístupy k testování výkonu React aplikací, např. pomocí React DevTools Profiler a Lighthouse.

## Knihy a oficiální dokumentace

1. **"High Performance Browser Networking"** – *Ilya Grigorik*  
   Detailní průvodce síťovými technologiemi a optimalizací webových aplikací, zaměřený na výkon při komunikaci mezi klientem a serverem.

2. **[MDN Web Performance Documentation](https://developer.mozilla.org/en-US/docs/Web/Performance)**  
   Oficiální dokumentace MDN o výkonu webu – zahrnuje teorii, nástroje a praktické doporučení pro optimalizaci výkonu.

3. **[The Performance Calendar](https://calendar.perfplanet.com/)** – *Performance Calendar*  
   Roční kalendář plný článků a příspěvků o optimalizaci výkonu, které mohou sloužit jako zdroj pro pokračující studium.

## Praktické projekty

1. **Vytvořit webovou stránku s optimalizovaným výkonem**  
   Vytvořte


## 📝 Shrnutí kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">✅ Co jste se naučili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>✓ Python profiling tools</li>
<li>✓ Memory profiling</li>
<li>✓ Caching strategies</li>
<li>✓ Redis implementation</li>
<li>✓ Database query optimization</li>
    </ul>
    
    <h3 style="color: white;">🎯 Klíčové dovednosti</h3>
    <p>Po dokončení této kapitoly byste měli být schopni prakticky použít všechny probrané koncepty.</p>
    
    <h3 style='color: white;'>➡️ Další kapitola</h3><p>Kapitola 27 - pokračujte ve studiu!</p>
</div>

---

*📅 Notebook vygenerován: 2025-09-29 12:57:34*  
*🤖 Generátor: Comprehensive Colab Generator v2.0*  
*📚 Učebnice programování - Od základů k AI*


## 🧪 Sandbox - Prostor pro experimenty

Použijte následující buňky pro vlastní experimenty a testování:

In [ ]:
# 🧪 Zde můžete experimentovat s kódem z kapitoly
# Napište svůj kód zde:

